# Proposal

Brad wanted the entire chip to be readable with one ROACH, so this notebook compresses the resonance frequencies into 500 MHz of bandwidth.

The current readout seems to have a sharp roll-off as the frequency approaches 4 GHz. Let's keep the highest resonance frequency below 3.8 GHz, neglecting kinetic inductance. This should allow us to test all-niobium resonators.

Split this band into four sub-bands. Since the expected loading is higher at 220 GHz and we probably need more active volume to keep $Q_i$ as high as possible, plan for the 220 bands to have longer active regions. To maintain approximately the same active-to-inactive ratio, place these bands at lower resonance frequencies.

Set the resonance spacing to 4 MHz, which is 10 linewidths for a $Q = 10^4$ resonator at 4 GHz. Each 23-resonator band will then occupy about 100 MHz. (We should really keep the fractional spacing constant, but I think we have enough bandwidth that this doesn't matter.) With 30 MHz gaps between the bands, we need about 130 MHz per 23-resonance band, or $4 \times 100 + 3 \times 30 = 490 \; \mathrm{MHz}$ total.

Since we don't know the kinetic inductance fractions a priori, and since they will vary somewhat between the bands and designs, start by writing the length as a function of desired resonance frequency.

In [1]:
from scipy.constants import c

The resonance frequency of a quarter-wavelength line resonator is approximately
$$
f_r = (1 - \alpha)^{1/2} \frac{c}{4 \ell \epsilon_e^{1/2}},
$$
where $\alpha$ is the kinetic inductance fraction, $c$ is the speed of light, $\ell$ is the length of the resonator, and $\epsilon_e$ is the effective dielectric constant. For a planar structure on a substrate with dielectric constant $\epsilon_s$, the effective dielectric constant is the average of the substrate and vacuum values:
$$
\epsilon_e = (1 + \epsilon_s) / 2.
$$

In [2]:
def resonance(length, alpha=0, epsilon_substrate=11.9):
    epsilon_effective = (1 + epsilon_substrate) / 2
    v = c / epsilon_effective**(1/2)
    wavelength = 4 * length
    f_r0 = v / wavelength
    f_r = (1 - alpha)**(1/2) * f_r0
    return f_r

def length(f_r, alpha=0, epsilon_substrate=11.9):
    epsilon_vacuum = 1
    epsilon_effective = (epsilon_vacuum + epsilon_substrate) / 2
    return (1 - alpha)**(1/2) * c / (4 * f_r * epsilon_effective**(1/2))

In [3]:
Q = 1e4
f_r = 4e9
df = f_r / Q
linewidths = 10
print("The nominal resonance spacing is {:.1f} MHz".format(1e-6 * df * linewidths))

The nominal resonance spacing is 4.0 MHz


In [4]:
resonators_per_band = 23
F = df * linewidths
B = F * resonators_per_band 
print("With {} resonators per band, each band occupies {:.0f} MHz".format(resonators_per_band, 1e-6 * B))

With 23 resonators per band, each band occupies 92 MHz


In [13]:
f_max = 3000e6  # The maximum resonance frequency
between = 30e6  # The frequency spacing between bands

In [14]:
high150 = np.linspace(f_max - B, f_max, resonators_per_band)
low150 = np.linspace(high150[0] - between - B, high150[0] - between, resonators_per_band)
high220 = np.linspace(low150[0] - between - B, low150[0] - between, resonators_per_band)
low220 = np.linspace(high220[0] - between - B, high220[0] - between, resonators_per_band)

In [15]:
print("The low-frequency 220 GHz band covers {:.0f} to {:.0f} MHz.".format(1e-6 * low220[0], 1e-6 * low220[-1]))
print("The high-frequency 220 GHz band covers {:.0f} to {:.0f} MHz.".format(1e-6 * high220[0], 1e-6 * high220[-1]))
print("The low-frequency 150 GHz band covers {:.0f} to {:.0f} MHz.".format(1e-6 * low150[0], 1e-6 * low150[-1]))
print("The high-frequency 150 GHz band covers {:.0f} to {:.0f} MHz.".format(1e-6 * high150[0], 1e-6 * high150[-1]))

The low-frequency 220 GHz band covers 2542 to 2634 MHz.
The high-frequency 220 GHz band covers 2664 to 2756 MHz.
The low-frequency 150 GHz band covers 2786 to 2878 MHz.
The high-frequency 150 GHz band covers 2908 to 3000 MHz.


In [16]:
1e3 * length(low220, alpha=0.2)

array([ 10.38365623,  10.36660224,  10.34960417,  10.33266176,
        10.31577473,  10.2989428 ,  10.28216572,  10.2654432 ,
        10.24877499,  10.23216082,  10.21560043,  10.19909356,
        10.18263995,  10.16623934,  10.14989147,  10.1335961 ,
        10.11735297,  10.10116182,  10.08502242,  10.06893451,
        10.05289784,  10.03691218,  10.02097727])

In [17]:
1e3 * length(high220, alpha=0.2)

array([ 9.90812843,  9.89259951,  9.87711918,  9.86168723,  9.84630343,
        9.83096755,  9.81567936,  9.80043865,  9.7852452 ,  9.77009878,
        9.75499918,  9.73994618,  9.72493956,  9.70997911,  9.69506463,
        9.68019589,  9.66537269,  9.65059481,  9.63586206,  9.62117422,
        9.60653108,  9.59193246,  9.57737813])

In [18]:
1e3 * length(low150, alpha=0.2)

array([ 9.47424772,  9.46004807,  9.44589093,  9.4317761 ,  9.41770339,
        9.40367261,  9.38968357,  9.3757361 ,  9.36183   ,  9.34796508,
        9.33414118,  9.3203581 ,  9.30661566,  9.2929137 ,  9.27925201,
        9.26563044,  9.2520488 ,  9.23850692,  9.22500462,  9.21154174,
        9.19811809,  9.1847335 ,  9.17138782])

In [19]:
1e3 * length(high150, alpha=0.2)

array([ 9.0767724 ,  9.06373839,  9.05074175,  9.03778234,  9.02485999,
        9.01197453,  8.99912582,  8.9863137 ,  8.973538  ,  8.96079858,
        8.94809528,  8.93542794,  8.92279642,  8.91020056,  8.89764021,
        8.88511523,  8.87262546,  8.86017075,  8.84775096,  8.83536594,
        8.82301554,  8.81069963,  8.79841805])

In [20]:
[1e6 * length(a).ptp() for a in [low220, high220, low150, high150]]

[405.48740109271932, 369.79007260212404, 338.6076611095329, 311.20962819849302]